In [176]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import copy


In [177]:
df = pd.read_csv('form.csv')

print(df) 

            Timestamp            Name Are you visiting for the first time?  \
0   3/31/2025 9:28:48      Rebecca Lu                                  NaN   
1    4/1/2025 7:57:16    Derek Liang                                   NaN   
2    4/1/2025 7:57:32     Logan golia                                  NaN   
3    4/1/2025 8:04:38     Daniel Kim                                   NaN   
4    4/1/2025 8:09:57      Kyle Hwang                                  NaN   
..                ...             ...                                  ...   
62  4/4/2025 22:08:58     Daniel Choi                                  NaN   
63  4/4/2025 22:35:19      audrey lee                                  NaN   
64  4/4/2025 22:38:41       Brian Seo                                  NaN   
65  4/4/2025 23:18:55  Nathanael Wang                                  NaN   
66                NaN      Jiwang Lee                                  NaN   

    House Church              Where would you like to be picked

In [178]:
class Driver:
    def __init__(self, name, amount_seats, pickup_location, service_type, plans):
        self.name = name
        self.amount_seats = amount_seats
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name


class Rider:
    def __init__(self, name, pickup_location, service_type, plans):
        self.name = name
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

In [179]:
# Maximum number of distinct locations a driver shoud drive to pick up their passengers
MAXIMUM_LOCATION_THRESHOLD = 2

# Default number of passengers the driver's car can hold.
PASSENGER_LIMIT = 4


In [180]:
people_list = df["Name"]
locations_list = df["Where would you like to be picked up?"]
service_list = df["Which service are you attending?"]
is_driver_list = df["Are you a driver?"]
plans_list = df["Preferred after church plans?"]

drivers = set()
riders = set()

for i in range(len(people_list)):
    # p = (is_driver_list[i])
    # print(type(p))
    if is_driver_list[i] == "Yes!":
        # print(is_driver_list[i])
        drivers.add(Driver(people_list[i], PASSENGER_LIMIT, locations_list[i], service_list[i], plans_list[i]))
    else:
        riders.add(Rider(people_list[i], locations_list[i], service_list[i], plans_list[i]))

# print("List of all drivers:\n")
# for d in drivers:
#     print(d.name)

In [181]:
def print_assignment(assignments): # driver to list of rider object
    for driver, riders in assignments.items():
        print(f"\nDriver: {driver.name} {driver.pickup_location.split()[0]}")
        for rider in riders:
            print(rider.name, rider.pickup_location.split()[0])

In [182]:
'''
Change amount of seats from the default number of 4 to another amount.
'''
for driver in drivers:
    if driver.name == "Ellen Kang":
        driver.amount_seats = 6

In [183]:
'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders = {
    "Oriana Tang": {"Grace Park"},
    "Daniel Choi": {"Jacob Lei"}
}

'''
Assign riders who should ride together
'''
rider_groups = [
    {"Josh Yang", "Austin Lee"},
]



In [184]:
def assign_riders_greedy(drivers, riders):
    print(len(riders))

    drivers = copy.deepcopy(drivers)
    riders = copy.deepcopy(riders)

    assignments = defaultdict(list)
    unassigned_riders = set(list(riders))

    # Group drivers and riders by pickup time
    riders_by_time = defaultdict(list)
    for rider in riders:
        riders_by_time[rider.service_type].append(rider)
    
    driver_name_to_object = dict()
    for driver in drivers:
        driver_name_to_object[driver.name] = driver

    rider_name_to_object = dict()
    for rider in riders:
        rider_name_to_object[rider.name] = rider

    # Assign passengers who need to ride with a specific driver
    for driver_name, required_rider_names in driver_required_riders.items():
        time = driver_name_to_object[driver_name].service_type
        possible_riders = riders_by_time[time]
        for rider_name in required_rider_names:
            rider = rider_name_to_object[rider_name]
            driver = driver_name_to_object[driver_name]
            if rider in unassigned_riders:
                assignments[driver].append(rider)
                unassigned_riders.remove(rider)
                driver.amount_seats -= 1
        # print_assignment(assignments)

    
    # Assign groups of riders.
    # List of list of riders objects
    rider_groups_obs = []
    for group in rider_groups:
        group_obs = [rider_name_to_object[rider_name] for rider_name in group]
        rider_groups_obs.append(group_obs)

    for group in rider_groups_obs:
        if all(r in unassigned_riders for r in group):
            for driver in drivers:
                if driver.amount_seats < len(group):
                    continue

                combined_group = assignments[driver] + group
                total_locations = {r.pickup_location for r in combined_group}
                if len(total_locations) <= MAXIMUM_LOCATION_THRESHOLD:
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    driver.amount_seats -= len(group)
                    break


    
    # Greedily assign passengers to drivers.
    for driver in drivers:
        # print(f'\nDriver {driver.name}')
       

        time = driver.service_type
        possible_riders = riders_by_time[time]

        found_set = False
        # Try forming groups of the amount that the passenger can hold all the way down to 0 passengers.
        for group_size in range(driver.amount_seats, 0, -1):
            if found_set:
                break
            for group in combinations(possible_riders, group_size):
                combined_group = list(group) + assignments[driver]
                locations = {r.pickup_location for r in combined_group} 
                if len(locations) <= MAXIMUM_LOCATION_THRESHOLD and all(r in unassigned_riders for r in group):
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    found_set = True
                    break  # Move to next driver
    print(f'unassigned riders: {[rider.name for rider in unassigned_riders]}')
    return assignments
x = assign_riders_greedy(drivers, riders)
# print(x)

52
unassigned riders: []


In [185]:
print_assignment(x)



Driver: Oriana Tang Off
Grace Park Life
Rachel Kim South
audrey lee South

Driver: Daniel Choi Off
Jacob Lei Off
JJ Lee Life
Ellie Jung  Life
Jeff Jiang Off

Driver: Jacob Deadlifts Off
Austin Lee Off
Josh Yang Off
Khang Le Off
Seojin Kwon Off

Driver: Clay Murphy Life
Daniel Kim  North
Nathanael Wang Life
Logan golia North
Phillip Seo Life

Driver: Olivia Chang Life
Rebecca Lu Off
helena song North
Gabriel Ni North
Darius Ajebon  North

Driver: Ric Chang Off
Joshua Lee Life
Grace Sowon Park  South
Jeffery Huang South
Ella lu South

Driver: Ellen Kang Life
Benjamin Kim South
Hannah Zhang South
Grace Kwon South
Joann South
Elise Cho South
Claire Doh South

Driver: Jocelyn Lee Off
Sehyun Jung Life
Karina Pan Off
Melody Hong Life
Christina Ko Life

Driver: Brian Seo South
Hannah Kim South
Jay Jung North
David Zhu South
Angel Koh North

Driver: Josh Paik Off
Derek Liang  South
Israel Haile South
Susanna Tang North
Jiwang Lee North

Driver: Lia Kim Off
Jane Yoo Off

Driver: Jack Havemann L

In [ ]:
'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders = {
    "Ellie Jung": {"JJ Lee"}
}

'''
Assign riders who should ride together
'''
rider_groups = [
]



In [199]:
def assign_riders_greedy_back(drivers, riders):
    print(len(riders))

    drivers = copy.deepcopy(drivers)
    riders = copy.deepcopy(riders)

    assignments = defaultdict(list)
    unassigned_riders = set(list(riders))

    # Group drivers and riders by pickup time
    riders_by_time = defaultdict(list)
    for rider in riders:
        riders_by_time[rider.plans].append(rider)
    
    driver_name_to_object = dict()
    for driver in drivers:
        driver_name_to_object[driver.name] = driver

    rider_name_to_object = dict()
    for rider in riders:
        rider_name_to_object[rider.name] = rider

    # Assign passengers who need to ride with a specific driver
    for driver_name, required_rider_names in driver_required_riders.items():
        time = driver_name_to_object[driver_name].plans
        possible_riders = riders_by_time[time]
        for rider_name in required_rider_names:
            rider = rider_name_to_object[rider_name]
            driver = driver_name_to_object[driver_name]
            if rider in unassigned_riders:
                assignments[driver].append(rider)
                unassigned_riders.remove(rider)
                driver.amount_seats -= 1
        # print_assignment(assignments)

    
    # Assign groups of riders.
    # List of list of riders objects
    rider_groups_obs = []
    for group in rider_groups:
        group_obs = [rider_name_to_object[rider_name] for rider_name in group]
        rider_groups_obs.append(group_obs)

    for group in rider_groups_obs:
        if all(r in unassigned_riders for r in group):
            for driver in drivers:
                if driver.amount_seats < len(group):
                    continue

                combined_group = assignments[driver] + group
                total_locations = {r.pickup_location for r in combined_group}
                if len(total_locations) <= MAXIMUM_LOCATION_THRESHOLD:
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    driver.amount_seats -= len(group)
                    break


    
    # Greedily assign passengers to drivers.
    for driver in drivers:
        # print(f'\nDriver {driver.name}')
       

        time = driver.plans
        possible_riders = riders_by_time[time]

        found_set = False
        # Try forming groups of the amount that the passenger can hold all the way down to 0 passengers.
        for group_size in range(driver.amount_seats, 0, -1):
            if found_set:
                break
            for group in combinations(possible_riders, group_size):
                combined_group = list(group) + assignments[driver]
                locations = {r.pickup_location for r in combined_group} 
                if len(locations) <= MAXIMUM_LOCATION_THRESHOLD and all(r in unassigned_riders for r in group):
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    found_set = True

    

                    break  # Move to next driver

    
    print(f'unassigned riders before using flexible: {[rider.name for rider in unassigned_riders]}')
    print("unassigned riders", unassigned_riders)

    # Assign drivers who are flexible here if there are unassigned riders
    plan_groups = defaultdict(list)
    for rider in unassigned_riders:
        plan_groups[rider.plans].append(rider)
    print(plan_groups)
    
    # for driver in drivers:
    #     # This driver is not flexible
    #     if "Flexible" not in driver.plans:
    #         continue
    #     # This flexible driver does not have any more space in the car.
    #     if driver.amount_seats <= 0:
    #         continue

    #     # Try to find put as many riders into a 'flexible' car
    #     for plan, riders_plans in plan_groups.items():
    #         # Filter only truly unassigned riders
    #         available_riders = [r for r in riders_plans if r in unassigned_riders]
    #         if not available_riders:
    #             continue # no more riders for these plans

    #         # Compute how many seats are actually left (in case earlier assignments forgot to update amount_seats correctly)
    #         seats_remaining = driver.amount_seats
    #         if seats_remaining <= 0:
    #             continue
    #         group_to_assign = available_riders[:seats_remaining]

    #         # Assign riders
    #         assignments[driver].extend(group_to_assign)
    #         print(driver.name, group_to_assign, driver.amount_seats)
    #         # print("fd", unassigned_riders)
    #         for rider in group_to_assign:
    #             unassigned_riders.remove(rider)
    #             plan_groups[plan].remove(rider)
            
    #         driver.amount_seats -= len(group_to_assign)
    #         print(f"[DEBUG] {driver.name} already has {len(assignments[driver])} riders, {driver.amount_seats} seats left")

    #         break
    for driver in drivers:
        if "Flexible" not in driver.plans:
            continue
        if driver.amount_seats <= 0:
            continue

        # Determine the plan of already assigned riders (if any)
        assigned_riders = assignments[driver]
        existing_plan = None
        if assigned_riders:
            existing_plan = assigned_riders[0].plans
            # Validate all already assigned riders have the same plan
            assert all(r.plans == existing_plan for r in assigned_riders), \
                f"[ERROR] Driver {driver.name} has mixed plans in existing assignments!"
        print("DDD", driver.name, driver.plans)
        for plan, riders_plans in plan_groups.items():
            if existing_plan is not None and plan != existing_plan:
                continue  # skip if not matching the existing plan

            available_riders = [r for r in riders_plans if r in unassigned_riders]
            if not available_riders:
                continue

            seats_remaining = driver.amount_seats
            if seats_remaining <= 0:
                continue

            group_to_assign = available_riders[:seats_remaining]

            if not group_to_assign:
                continue

            # Assign
            assignments[driver].extend(group_to_assign)
            for rider in group_to_assign:
                unassigned_riders.remove(rider)
                plan_groups[plan].remove(rider)

            driver.amount_seats -= len(group_to_assign)
            print(f"[DEBUG] {driver.name} got {len(group_to_assign)} riders of plan '{plan}' — now has {len(assignments[driver])} total")
            break  # move to next flexible driver
            

    print(f'unassigned riders after using flexible: {[rider.name for rider in unassigned_riders]}')

    return assignments
x = assign_riders_greedy_back(drivers, riders)
# print(x)

52
unassigned riders before using flexible: ['Seojin Kwon', 'Maya Habraken ', 'Melody Hong', 'Grace Park', 'chloe lim', 'JJ Lee', 'Jacob Lei', 'Ellie Jung ', 'Kyle Hwang', 'Jeff Jiang']
unassigned riders {<__main__.Rider object at 0x7f8718bfab50>, <__main__.Rider object at 0x7f87490e6cd0>, <__main__.Rider object at 0x7f8718311cd0>, <__main__.Rider object at 0x7f8718235b80>, <__main__.Rider object at 0x7f8718bf6940>, <__main__.Rider object at 0x7f8718bf63d0>, <__main__.Rider object at 0x7f8718bf6730>, <__main__.Rider object at 0x7f8718bf6be0>, <__main__.Rider object at 0x7f8718bf6700>, <__main__.Rider object at 0x7f8718bf6760>}
defaultdict(<class 'list'>, {'NLK 🧡': [<__main__.Rider object at 0x7f8718bfab50>], 'Refreshments 💜 (Phnom Penh HC only)': [<__main__.Rider object at 0x7f87490e6cd0>], 'Back home 💙': [<__main__.Rider object at 0x7f8718311cd0>, <__main__.Rider object at 0x7f8718235b80>, <__main__.Rider object at 0x7f8718bf6940>, <__main__.Rider object at 0x7f8718bf63d0>, <__main__.

In [200]:
print_assignment(x)


Driver: Jacob Deadlifts Off
Nathanael Wang Life
Phillip Seo Life
Josh Yang Off
Sehyun Jung Life

Driver: Clay Murphy Life
Daniel Kim  North
Logan golia North
Jane Yoo Off
Darius Ajebon  North

Driver: Olivia Chang Life
helena song North
Jeffery Huang South
Elise Cho South
Susanna Tang North

Driver: Ric Chang Off
Rebecca Lu Off
Benjamin Kim South
Joann South

Driver: Oriana Tang Off
Gabriel Ni North

Driver: Ellen Kang Life
Grace Sowon Park  South
Ella lu South
Hannah Zhang South
Rachel Kim South
Karina Pan Off
Hannah Kim South

Driver: Jocelyn Lee Off
Christina Ko Life
Austin Lee Off
Aaron Long Life
Ethan Yu Life

Driver: Brian Seo South
Khang Le Off
Claire Doh South
Sam Ko  Off
Taeho Choe  South

Driver: Josh Paik Off
Seojin Kwon Off

Driver: Lia Kim Off
Jay Jung North
David Zhu South
Angel Koh North
Derek Liang  South

Driver: Jack Havemann Life

Driver: Matthew Ahn Off
Emily Yang North

Driver: Daniel Choi Off
Israel Haile South
Jiwang Lee North
Daniel Kuo South
Aimy Pham North

D

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill

# Create a new workbook and select the active sheet
wb = Workbook()
ws = wb.active
ws.title = "My Sheet"

# Write text into specific cells
ws["A1"] = "Name"
ws["B1"] = "Score"
ws["A2"] = "Alice"
ws["B2"] = 95
ws["A3"] = "Bob"
ws["B3"] = 88

# Apply bold font to headers
bold_font = Font(bold=True)
ws["A1"].font = bold_font
ws["B1"].font = bold_font

# Center align header text
center_alignment = Alignment(horizontal="center")
ws["A1"].alignment = center_alignment
ws["B1"].alignment = center_alignment

# Set background fill color for header
header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
ws["A1"].fill = header_fill
ws["B1"].fill = header_fill

# Save to file
wb.save("formatted_output.xlsx")